# 0. Import dependencies

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

# 1. Create the Environment

In [5]:
class ShowerEnv(Env):
        def __init__(self):
                # Actions we can take: down, stay, up
                self.action_space = Discrete(3)

                # Temperature array
                self.observation_space = Box(low=np.array([0]), high=np.array([100]))

                # Set start temp
                self.state = 38 + random.randint(-3,3)

                # Set shower length
                self.shower_length = 60
                 
        def step(self, action):
                # Apply action
                self.state += action - 1
                
                # Reduce shower length by 1 second
                self.shower_length -= 1

                # Calculate reward
                if self.state >= 37 and self.state <= 39:
                        reward = 1
                else:
                        reward = -1
                
                # Check if shower is done
                if self.shower_length <= 0:
                        done = True
                else:
                        done = False
                
                # Apply temperature noise
                self.state += random.randint(-1,1)

                # Set placeholder for info
                info = {}

                # Return step information
                return self.state, reward, done, info

        def render(self):
                # Implement visualization
                pass

        def reset(self):
                # Reset shower temperature
                self.state = 38 + random.randint(-3,3)

                # Reset shower time
                self.shower_length = 60

                return self.state

In [6]:
env = ShowerEnv()

C:\Users\bogda\anaconda3\envs\rl\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [28]:
env.observation_space.sample()

array([34.437687], dtype=float32)

In [29]:
episodes = 10
for episode in range (1, episodes + 1):
        state = env.reset()
        done = False
        score = 0

        while not done:
                #env.render()
                action = env.action_space.sample()
                n_state, reward, done, info = env.step(action)
                score += reward
        print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:-26
Episode:2 Score:-58
Episode:3 Score:-40
Episode:4 Score:-40
Episode:5 Score:30
Episode:6 Score:-50
Episode:7 Score:-28
Episode:8 Score:-30
Episode:9 Score:-48
Episode:10 Score:-52


# 2. Keras Deep Learning Model

In [39]:
import tensorflow
#from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [40]:
states = env.observation_space.shape
actions = env.action_space.n

In [41]:
def build_model(states, actions):
        model = tensorflow.keras.Sequential()
        model.add(Dense(24, activation='relu', input_shape=states))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(actions, activation='linear'))
        return model

In [42]:
model = build_model(states, actions)

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [44]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [45]:
def build_agent(model, actions):
        policy = BoltzmannQPolicy()
        memory = SequentialMemory(limit=50000, window_length=1)
        dqn = DQNAgent(model=model, memory=memory, policy=policy,
                nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
        return dqn

In [46]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 6:47 - reward: 1.0000

C:\Users\bogda\anaconda3\envs\rl\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\bogda\anaconda3\envs\rl\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 54s 5ms/step - reward: -0.7574
166 episodes - episode_reward: -45.398 [-60.000, 6.000] - loss: 2.320 - mae: 9.934 - mean_q: -10.612

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 55s 6ms/step - reward: -0.6170
167 episodes - episode_reward: -37.234 [-60.000, 26.000] - loss: 3.137 - mae: 13.186 - mean_q: -18.950

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: -0.6004
167 episodes - episode_reward: -35.892 [-60.000, 22.000] - loss: 3.372 - mae: 14.322 - mean_q: -20.632

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: -0.6086
166 episodes - episode_reward: -36.723 [-60.000, 14.000] - loss: 2.958 - mae: 13.416 - mean_q: -19.289

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 51s 5ms/step - reward: -0.6530
done, took 262.469 seconds


In [47]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -48.000, steps: 60
Episode 2: reward: -56.000, steps: 60
Episode 3: reward: -14.000, steps: 60
Episode 4: reward: -22.000, steps: 60
Episode 5: reward: -56.000, steps: 60
Episode 6: reward: -44.000, steps: 60
Episode 7: reward: -20.000, steps: 60
Episode 8: reward: -40.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -22.000, steps: 60
Episode 11: reward: -34.000, steps: 60
Episode 12: reward: -12.000, steps: 60
Episode 13: reward: 10.000, steps: 60
Episode 14: reward: -16.000, steps: 60
Episode 15: reward: -34.000, steps: 60
Episode 16: reward: -12.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: -48.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -58.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -28.000, steps: 60
Episode 24: reward: -4.000, steps: 60
Episode 25: reward: 32.000, steps: 60
Episode 